In [169]:
import pandas as pd
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from scikeras.wrappers import KerasClassifier, KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras import Sequential
from keras.layers import Dense, Dropout
import keras.metrics
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, TensorBoard
from MLscores import calc_metrics, metrics_dict, cmvals, recall, hybridrecall
import keras.backend as K
import tensorflow as tf
from functools import partial
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import recall_score, make_scorer
from xgboost import XGBClassifier
from check_and_prepare_dataset import load_dataset, prepare_dataset
from manage_model import create_NN_model, create_sklearn_model, allowgrowthgpus
import re
import numpy as np
#import eli5
#from eli5.sklearn import PermutationImportance

In [170]:
#params={'ES_mindelta': 0.001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': 0.1, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 70}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params={'ES_mindelta': 0.0001, 'ES_monitor': 'loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': 0.3, 'feature_drop': ('dir_max', 'dom_dir', 'month', 'wkd'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 200.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params = {}
#params['nn_auc']= {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params['nn_f1']={'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 2}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 60.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
params['nn_rh5']={'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1300.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params['nn_nh5'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 10}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1100.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params['nn_h1'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 2}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 60.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params['nn_h2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_gr1', 'corine_gr2', 'corine_gr3', 'corine_gr4', 'corine_gr5', 'corine_gr6', 'corine_gr7', 'corine_gr8', 'corine_gr9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}
#params['nn_nh2'] = {'ES_mindelta': 0.002, 'ES_monitor': 'val_loss', 'ES_patience': 10, 'batch_size': 512, 'class_weights': {0: 1, 1: 5}, 'dropout': None, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'pop', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_epochs': 2000, 'metric': 'accuracy', 'n_internal_layers': (0, {'layer_1_0_nodes': 1700.0}), 'optimizer': {'adam_params': None, 'name': 'Adam'}}


In [171]:
models={}
for paramset in params:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv', \
                         params[paramset]['feature_drop']+ \
                         ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp')) #correlated features
    creatennmodel = partial(create_NN_model, params[paramset], X)
    models[paramset]={}
    models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)
    models[paramset]['params'] = params[paramset]

Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv
before nan drop: 34967
after nan drop: 34967
after dup. drop: 34880
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate', 'id']
Dropped columns ['bin_weekday_4', 'dom_vel', 'bin_corine_19', 'bin_corine_24', 'bin_month_9', 'bin_weekday_7', 'bin_corine_43', 'bin_corine_4', 'bin_corine_22', 'bin_month_8', 'bin_dir_max_4', 'bin_dir_max_7', 'bin_corine_35', 'bin_corine_28', 'bin_corine_9', 'bin_corine_2', 'bin_corine_33', 'bin_corine_32', 'bin_weekday_5', 'bin_month_10', 'bin_corine_13', 'bin_corine_27', 'bin_corine_7', 'bin_corine_37', 'bin_corine_18', 'min_temp', 'bin_corine_3', 'bin_month_3', 'bin_corine_20', 'bin_month_4', 'bin_corine_40', 'bin_corine_44', 'min_dew_temp', 'bin_corine_12', 'bin_dir_max_5', 'bin_month_5', 'bin_corine_36', 'evi', 'bin_corine_42', 'bin_dir_max_2', 'max_temp', 'bin_dir_max_3',

/tmp/ipykernel_2561625/2955017594.py:8: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  models[paramset]['model'] = KerasClassifier(build_fn=creatennmodel, batch_size=params[paramset]['batch_size'], epochs=300, verbose=0,)


In [179]:
paramssk={}
#RF best BA - rh2
paramssk['RF_rh2']={'algo': 'RF', 'bootstrap': False, 'class_weights': {0: 1, 1: 10}, 'criterion': 'entropy', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': 20, 'max_features': 1.0, 'min_samples_leaf': 30, 'min_samples_split': 10, 'n_estimators': 150}
#ET best BA- sh1
paramssk['ET_sh1']={'algo': 'XT', 'bootstrap': False, 'class_weights': {0: 1, 1: 10}, 'criterion': 'gini', 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'max_depth': 50, 'max_features': 4.0, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 800}
#XGB best -rh2
paramssk['XGB_rh2']={'algo': 'XGB', 'alpha': 100, 'feature_drop': ('month', 'weekday', 'dom_dir', 'dir_max', 'corine_1', 'corine_2', 'corine_3', 'corine_4', 'corine_5', 'corine_6', 'corine_7', 'corine_8', 'corine_9'), 'gamma': 0.001, 'lambda': 10.0, 'max_depth': 6.0, 'n_estimators': 320, 'scale_pos_weight': 70, 'subsample': 0.9}
for skalgo in paramssk:
    print(paramssk[skalgo]['algo'])
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv', \
                         params[paramset]['feature_drop'] +
                        ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp'))
    models[skalgo]={}
    models[skalgo]['model'] = create_sklearn_model(paramssk[skalgo],X)
    models[skalgo]['params']=paramssk[skalgo]


RF
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_norm.csv
before nan drop: 41771
after nan drop: 41771
after dup. drop: 41684
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_weekday_4', 'dom_vel', 'bin_corine_19', 'bin_corine_24', 'bin_month_9', 'bin_weekday_7', 'bin_corine_43', 'bin_corine_4', 'bin_corine_22', 'bin_month_8', 'bin_dir_max_4', 'bin_dir_max_7', 'bin_corine_35', 'bin_corine_28', 'bin_corine_9', 'bin_corine_2', 'bin_corine_33', 'bin_corine_32', 'bin_weekday_5', 'bin_month_10', 'bin_corine_13', 'bin_corine_27', 'bin_corine_7', 'bin_corine_37', 'bin_corine_18', 'min_temp', 'bin_corine_3', 'bin_month_3', 'bin_corine_20', 'bin_month_4', 'bin_corine_40', 'bin_corine_44', 'min_dew_temp', 'bin_corine_12', 'bin_dir_max_5', 'bin_month_5', 'bin_corine_36', 'evi', 'bin_corine_42', 'bin_dir_max_2', 'max_temp', 'bin_dir_max_3', 'bin_cori

In [180]:
#models1={'RF':models['RF'], 'ET':models['ET']}
allowgrowthgpus()
for modname in models:
    X, y, g=load_dataset('/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv', \
                         models[modname]['params']['feature_drop']+ \
                         ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp', 'pop'))
    print(modname)
    models[modname]['model'].fit(X,y)
    models[modname]['columns']=X.columns
    

1 Physical GPUs, 1 Logical GPUs
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv
before nan drop: 34967
after nan drop: 34967
after dup. drop: 34880
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate', 'id']
Dropped columns ['bin_weekday_4', 'dom_vel', 'bin_corine_19', 'bin_corine_24', 'bin_month_9', 'bin_weekday_7', 'bin_corine_43', 'bin_corine_4', 'bin_corine_22', 'bin_month_8', 'bin_dir_max_4', 'bin_dir_max_7', 'bin_corine_35', 'bin_corine_28', 'bin_corine_9', 'bin_corine_2', 'bin_corine_33', 'bin_corine_32', 'bin_weekday_5', 'bin_month_10', 'bin_corine_13', 'bin_corine_27', 'bin_corine_7', 'bin_corine_37', 'bin_corine_18', 'min_temp', 'bin_corine_3', 'bin_month_3', 'bin_corine_20', 'bin_month_4', 'bin_corine_40', 'bin_corine_44', 'min_dew_temp', 'bin_corine_12', 'bin_dir_max_5', 'bin_month_5', 'bin_corine_36', 'evi', 'bin_corine_42', 'bin_dir_max_

/tmp/ipykernel_2561625/648497118.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  models[modname]['model'].fit(X,y)


Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv
before nan drop: 34967
after nan drop: 34967
after dup. drop: 34880
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate', 'id']
Dropped columns ['bin_weekday_4', 'dom_vel', 'bin_corine_19', 'bin_corine_24', 'bin_month_9', 'bin_weekday_7', 'bin_corine_43', 'bin_corine_4', 'bin_corine_22', 'bin_month_8', 'bin_dir_max_4', 'bin_dir_max_7', 'bin_corine_35', 'bin_corine_28', 'bin_corine_9', 'bin_corine_2', 'bin_corine_33', 'bin_corine_32', 'bin_weekday_5', 'bin_month_10', 'bin_corine_13', 'bin_corine_27', 'bin_corine_7', 'bin_corine_37', 'bin_corine_18', 'min_temp', 'bin_corine_3', 'bin_month_3', 'bin_corine_20', 'bin_month_4', 'bin_corine_40', 'bin_corine_44', 'min_dew_temp', 'bin_corine_12', 'bin_dir_max_5', 'bin_month_5', 'bin_corine_36', 'evi', 'bin_corine_42', 'bin_dir_max_2', 'max_temp', 'bin_dir_max_3',

/tmp/ipykernel_2561625/648497118.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  models[modname]['model'].fit(X,y)


In [182]:
def perm_results(models, dataset):
    rec1_scorer = make_scorer(recall_score, pos_label=1)
    rec0_scorer = make_scorer(recall_score, pos_label=0)
    scoring={'recall1':rec1_scorer, 'recall0':rec0_scorer}
    for modname in models:
        print(modname)
        #models[modname].fit(X,y)
        #ypred=models['nn_auc'].predict(X)
        X, y, g=load_dataset(dataset, \
                              models[modname]['params']['feature_drop']+ \
                             ('evi','dom_vel', 'max_dew_temp', 'min_dew_temp', 'max_temp', 'min_temp', 'pop'))
        r=permutation_importance(models[modname]['model'], X, y, n_repeats=3, scoring=scoring)
        models[modname]['permresults']=r

In [183]:
perm_results(models, '/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv')

nn_rh5
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv
before nan drop: 34967
after nan drop: 34967
after dup. drop: 34880
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate', 'id']
Dropped columns ['bin_weekday_4', 'dom_vel', 'bin_corine_19', 'bin_corine_24', 'bin_month_9', 'bin_weekday_7', 'bin_corine_43', 'bin_corine_4', 'bin_corine_22', 'bin_month_8', 'bin_dir_max_4', 'bin_dir_max_7', 'bin_corine_35', 'bin_corine_28', 'bin_corine_9', 'bin_corine_2', 'bin_corine_33', 'bin_corine_32', 'bin_weekday_5', 'bin_month_10', 'bin_corine_13', 'bin_corine_27', 'bin_corine_7', 'bin_corine_37', 'bin_corine_18', 'min_temp', 'bin_corine_3', 'bin_month_3', 'bin_corine_20', 'bin_month_4', 'bin_corine_40', 'bin_corine_44', 'min_dew_temp', 'bin_corine_12', 'bin_dir_max_5', 'bin_month_5', 'bin_corine_36', 'evi', 'bin_corine_42', 'bin_dir_max_2', 'max_temp', 'bin_dir_

ET_sh1
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv
before nan drop: 34967
after nan drop: 34967
after dup. drop: 34880
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate', 'id']
Dropped columns ['bin_weekday_4', 'dom_vel', 'bin_corine_19', 'bin_corine_24', 'bin_month_9', 'bin_weekday_7', 'bin_corine_43', 'bin_corine_4', 'bin_corine_22', 'bin_month_8', 'bin_dir_max_4', 'bin_dir_max_7', 'bin_corine_35', 'bin_corine_28', 'bin_corine_9', 'bin_corine_2', 'bin_corine_33', 'bin_corine_32', 'bin_weekday_5', 'bin_month_10', 'bin_corine_13', 'bin_corine_27', 'bin_corine_7', 'bin_corine_37', 'bin_corine_18', 'min_temp', 'bin_corine_3', 'bin_month_3', 'bin_corine_20', 'bin_month_4', 'bin_corine_40', 'bin_corine_44', 'min_dew_temp', 'bin_corine_12', 'bin_dir_max_5', 'bin_month_5', 'bin_corine_36', 'evi', 'bin_corine_42', 'bin_dir_max_2', 'max_temp', 'bin_dir_

In [201]:
def getcorinedesc(corinefeat, dfcorcats, typ='all'):
    #print(corinefeat)
    if not 'corine' in corinefeat:
        return corinefeat
    r=re.search(r"\d+$",corinefeat)
    corid=r.group(0)
    if not 'gr' in corinefeat:
        corstr=dfcorcats.loc[dfcorcats['Unnamed: 3']==corid,'CORINE LAND COVER LEGEND'].values[0]
    else:
        if len(corid)==1:
            corstr=dfcorcats.loc[dfcorcats['Unnamed: 0'].str[0]==corid,'Unnamed: 0'].values[0]

        elif len(corid)==2:
            coriddot=corid[0]+'.'+corid[1]
            corstr=dfcorcats.loc[dfcorcats['Unnamed: 1'].str[0:3]==coriddot,'Unnamed: 1'].values[0]
    if typ=='num':
        corstr='cor. cat. '+re.search(r"^[\.0-9]*",corstr).group(0)
    return corstr

def colcond1(columns,i):
    return not any(c in columns[i] for c in ['dir_max','dom_dir','wkd','month','corine'])

def colcond0(columns,i):
    return not any(c in columns[i] for c in ['dir_max','dom_dir','wkd','month'])

def colcond2(columns,i):
    return 'corine' in columns[i]
    
#for r in modelsresults
def resdict(condfunc, models, modname):
    modelsresult=models[modname]['permresults']
    columns=models[modname]['columns']
    results_lists={}
    for s in modelsresult:
        results_lists['%s %s'%(modname,s)]=[]
        cnt=0
        for i in modelsresult[s].importances_mean.argsort()[::-1]:
            #if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
            cnt+=1
            if condfunc(columns,i):
                results_dict={}
                results_dict['rank %s %s'%(modname,s)]=cnt
                results_dict['feat %s %s'%(modname,s)]=columns[i]
                results_dict['perm. imp. %s %s'%(modname,s)]='%.1f%%'%round(modelsresult[s].importances_mean[i]*100,1)
                results_lists['%s %s'%(modname,s)]+=[results_dict]
    return results_lists

def get_perm_pandas(cond, models, modname):
    results_lists=resdict(cond, models, modname)
    results_pd=[]
    for mod in results_lists:
        results_pd+=[pd.DataFrame(results_lists[mod])]
    all_res_pd=pd.concat(results_pd, axis=1)
    return all_res_pd

def get_perm_pandas_all_models(cond, models, nlines=15):
    permpds=[]
    for modname in models:
        permpds+=[get_perm_pandas(cond, models, modname).iloc[:nlines,:]]
    all_mod_df=pd.concat(permpds, axis=1)
    ranks=[c for c in all_mod_df.columns if 'rank' in c]
    all_mod_df.rename(columns={ranks[0]:'rank'},inplace=True)
    all_mod_df['rank'].astype(np.int32)
    all_mod_df.drop(columns=ranks[1:],inplace=True)
    return all_mod_df

In [203]:
pres=get_perm_pandas_all_models(colcond0, models, 30)
for modname in models:
    pres['feat %s recall0'%modname]=pres['feat %s recall0'%modname].apply(getcorinedesc, args=(dfcorcats,'num'))
    pres['feat %s recall1'%modname]=pres['feat %s recall1'%modname].apply(getcorinedesc, args=(dfcorcats,'num'))
pres#.to_csv('/mnt/nvme2tb/ffp/results/best2/permimpall.csv', index=False)

,rank,feat nn_rh5 recall1,perm. imp. nn_rh5 recall1,feat nn_rh5 recall0,perm. imp. nn_rh5 recall0,feat XGB_rh2 recall1,perm. imp. XGB_rh2 recall1,feat XGB_rh2 recall0,perm. imp. XGB_rh2 recall0,feat ET_sh1 recall1,perm. imp. ET_sh1 recall1,feat ET_sh1 recall0,perm. imp. ET_sh1 recall0
0,1,res_max,23.8%,res_max,5.5%,res_max,7.5%,res_max,5.9%,res_max,8.0%,cor. cat. 3.2,4.4%
1,2,f81,15.1%,f81,5.2%,xpos,1.4%,mean_dew_temp,3.6%,f81,1.3%,f81,4.1%
2,3,mean_temp,12.5%,cor. cat. 3.2,4.8%,mean_dew_temp,1.2%,xpos,3.2%,ypos,0.8%,res_max,4.1%
3,4,ypos,11.4%,mean_dew_temp,3.9%,ypos,1.1%,f81,2.9%,lst_day,0.7%,dem,2.2%
4,5,dem,11.2%,ypos,3.7%,f81,0.9%,dem,2.3%,rain_7_days,0.7%,mean_dew_temp,1.8%
5,6,xpos,9.2%,xpos,3.1%,mean_temp,0.6%,ypos,1.9%,mean_temp,0.7%,ypos,1.7%
6,7,ndvi,8.9%,cor. cat. 2.1,2.7%,rain_7_days,0.5%,lst_day,1.5%,cor. cat. 3.2,0.6%,frequency,1.2%
7,8,mean_dew_temp,7.6%,dem,2.6%,dem,0.3%,lst_night,1.0%,dem,0.6%,xpos,1.0%
8,9,cor. cat. 3.1,6.9%,cor. cat. 2.2,2.4%,lst_day,0.3%,cor. cat. 3.2,0.9%,ndvi,0.4%,cor. cat. 2.2,0.9%
9,10,cor. cat. 3.2,6.4%,cor. cat. 2.4,1.8%,ndvi,0.2%,mean_temp,0.9%,cor. cat. 2.1,0.3%,mean_temp,0.9%


In [158]:
re.match(r"^[\.0-9]*",'3.4 fff')

<re.Match object; span=(0, 3), match='3.4'>

In [55]:
m1=next(iter(models.items()))
permres=get_perm_results({m1[0]:m1[1]}, '/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv')

nn_f1
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_greece_norm.csv
before nan drop: 34967
after nan drop: 34967
after dup. drop: 34880
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate', 'id']
Dropped columns ['bin_dir_max_8', 'bin_weekday_4', 'dom_vel', 'bin_dom_dir_7', 'bin_weekday_1', 'bin_month_9', 'bin_dom_dir_6', 'bin_weekday_7', 'pop', 'bin_month_8', 'bin_dir_max_6', 'bin_dir_max_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_dir_max_7', 'bin_corine_gr1', 'bin_weekday_5', 'bin_weekday_3', 'bin_month_10', 'bin_corine_gr21', 'bin_corine_gr22', 'bin_dom_dir_8', 'min_temp', 'bin_month_3', 'bin_corine_gr24', 'bin_dom_dir_3', 'bin_dom_dir_5', 'bin_corine_gr33', 'bin_dom_dir_2', 'bin_month_4', 'min_dew_temp', 'bin_corine_gr31', 'bin_dir_max_5', 'bin_dir_max_1', 'bin_month_5', 'bin_corine_gr4', 'max_dew_temp', 'bin_month_6', 'bin_corine_gr5', 'evi', 'bin_month_7', 

1090/1090 [==============================] - 0s 310us/step


In [123]:
get_perm_pandas(colcond0, models, 'nn_f1')

,rank nn_f1 recall1,feat nn_f1 recall1,perm. imp. nn_f1 recall1,rank nn_f1 recall0,feat nn_f1 recall0,perm. imp. nn_f1 recall0
0,1,res_max,20.4%,1,res_max,5.0%
1,2,mean_temp,8.0%,2,mean_dew_temp,3.3%
2,3,f81,8.0%,3,dem,2.3%
3,4,dem,5.8%,4,bin_corine_17,2.1%
4,5,bin_corine_23,4.2%,5,xpos,1.9%
5,6,bin_corine_28,3.9%,6,bin_corine_12,1.9%
6,7,xpos,3.3%,7,bin_corine_33,1.6%
7,8,ypos,3.3%,8,bin_corine_13,1.5%
8,9,mean_dew_temp,2.9%,9,f81,1.5%
9,10,ndvi,2.7%,10,bin_corine_26,1.3%


In [141]:
modname='nn_rh5'
if any('ine_gr' in c for c in models[modname]['columns']):
    cond=colcond0
else:
    cond=colcond1
res_pd=get_perm_pandas(colcond0, models, modname)
res_pd['feat %s recall0'%modname]=res_pd['feat %s recall0'%modname].apply(getcorinedesc, args=(dfcorcats,'num'))
res_pd['feat %s recall1'%modname]=res_pd['feat %s recall1'%modname].apply(getcorinedesc, args=(dfcorcats,'num'))
res_pd

,rank nn_rh5 recall1,feat nn_rh5 recall1,perm. imp. nn_rh5 recall1,rank nn_rh5 recall0,feat nn_rh5 recall0,perm. imp. nn_rh5 recall0
0,1,res_max,25.8%,1,res_max,4.8%
1,2,f81,17.4%,2,f81,3.6%
2,3,mean_temp,13.7%,3,cor. cat.3.2,3.6%
3,4,ypos,12.8%,4,mean_dew_temp,3.5%
4,5,dem,11.1%,5,ypos,3.2%
5,6,xpos,10.3%,6,xpos,2.7%
6,7,ndvi,9.1%,7,cor. cat.2.1,2.3%
7,8,road_dens,7.1%,8,dem,2.1%
8,9,mean_dew_temp,6.9%,9,cor. cat.2.4,1.8%
9,10,cor. cat.3.2,6.7%,10,cor. cat.2.2,1.8%


In [139]:
modname='nn_rh5'
results_lists=resdict(colcond2, models, modname)
results_pd=[]
for mod in results_lists:
    results_pd+=[pd.DataFrame(results_lists[mod])]
all_res_pd2=pd.concat(results_pd, axis=1)
dfcorcats=pd.read_csv('/mnt/nvme2tb/ffp/code/mlfires/ML_fires_al/corine_categories.csv')
all_res_pd2['feat %s recall0'%modname]=all_res_pd2['feat %s recall0'%modname].apply(getcorinedesc, args=(dfcorcats,'num'))
all_res_pd2['feat %s recall1'%modname]=all_res_pd2['feat %s recall1'%modname].apply(getcorinedesc, args=(dfcorcats,'num'))

In [140]:
all_res_pd2

,rank nn_rh5 recall1,feat nn_rh5 recall1,perm. imp. nn_rh5 recall1,rank nn_rh5 recall0,feat nn_rh5 recall0,perm. imp. nn_rh5 recall0
0,10,cor. cat.3.2,6.7%,3,cor. cat.3.2,3.6%
1,11,cor. cat.3.1,6.6%,7,cor. cat.2.1,2.3%
2,13,cor. cat.3.3,3.7%,9,cor. cat.2.4,1.8%
3,14,cor. cat.2.1,3.6%,10,cor. cat.2.2,1.8%
4,15,cor. cat.2.4,3.5%,15,cor. cat.3.1,0.8%
5,18,cor. cat.2.2,2.0%,17,cor. cat.3.3,0.6%
6,19,cor. cat.1.,1.5%,19,cor. cat.1.,0.4%
7,20,cor. cat.2.3,1.2%,20,cor. cat.2.3,0.2%
8,22,cor. cat.5.,0.8%,22,cor. cat.5.,0.1%
9,23,cor. cat.4.,0.6%,24,cor. cat.4.,0.0%


In [23]:
all_res_pd.to_csv('/mnt/nvme2tb/ffp/results/newdefCV/permimpcorine.csv')
all_res_pd2.to_csv('/mnt/nvme2tb/ffp/results/newdefCV/permimpmainfeat.csv')

In [96]:
models['nn_rh5']['columns']

Index(['bin_corine_gr23', 'bin_corine_gr22', 'rain_7_days', 'bin_corine_gr5',
       'slope', 'bin_corine_gr33', 'frequency', 'bin_corine_gr31', 'road_dens',
       'lst_day', 'bin_corine_gr4', 'aspect', 'ndvi', 'bin_corine_gr32',
       'ypos', 'bin_corine_gr1', 'res_max', 'dem', 'xpos', 'mean_dew_temp',
       'f81', 'bin_corine_gr24', 'lst_night', 'bin_corine_gr21', 'mean_temp'],
      dtype='object')

In [39]:
m1=next(iter(models.items()))
modres=get_perm_results({m1[0]:m1[1]}, '/mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_attica_norm.csv')

nn_h1
Loading full dataset /mnt/nvme2tb/ffp/datasets/train/train_new_sample_1_2_attica_norm.csv
before nan drop: 2055
after nan drop: 2055
after dup. drop: 2054
renaming "x": "xpos", "y": "ypos"
Ignored columns from csv ['index', 'y', 'x', 'dom_dir', 'dir_max', 'fire', 'weekday', 'month', 'band', 'firedate']
Dropped columns ['bin_dir_max_8', 'bin_weekday_4', 'dom_vel', 'bin_dom_dir_7', 'bin_weekday_1', 'bin_month_9', 'bin_dom_dir_6', 'bin_weekday_7', 'bin_month_8', 'bin_dir_max_6', 'bin_dir_max_4', 'bin_weekday_2', 'bin_corine_gr32', 'bin_dir_max_7', 'bin_corine_gr1', 'bin_weekday_5', 'bin_weekday_3', 'bin_month_10', 'bin_corine_gr21', 'bin_corine_gr22', 'bin_dom_dir_8', 'min_temp', 'bin_month_3', 'bin_corine_gr24', 'bin_dom_dir_3', 'bin_dom_dir_5', 'bin_corine_gr33', 'bin_dom_dir_2', 'bin_month_4', 'min_dew_temp', 'bin_corine_gr31', 'bin_dir_max_5', 'bin_dir_max_1', 'bin_month_5', 'bin_corine_gr4', 'max_dew_temp', 'bin_month_6', 'bin_corine_gr5', 'evi', 'bin_month_7', 'bin_corine_gr23

65/65 [==============================] - 0s 374us/step


In [40]:
dfresgen=get_perm_pandas(colcond1, modres[0])
dfresgen

,rank nn_h1 recall1,feat nn_h1 recall1,perm. imp. nn_h1 recall1,rank nn_h1 recall0,feat nn_h1 recall0,perm. imp. nn_h1 recall0
0,1,f81,10.9%,1,res_max,13.7%
1,2,pop,7.2%,2,mean_temp,6.8%
2,3,res_max,6.4%,3,dem,3.7%
3,6,ypos,3.4%,10,pop,1.5%
4,7,mean_dew_temp,3.1%,11,mean_dew_temp,1.2%
5,9,mean_temp,2.7%,12,ndvi,1.1%
6,11,road_dens,1.9%,15,road_dens,0.8%
7,15,xpos,1.4%,16,frequency,0.8%
8,19,dem,0.9%,21,lst_day,0.4%
9,20,aspect,0.8%,28,lst_night,0.2%


In [42]:
dfrescor=get_perm_pandas(colcond2, modres[0])
dfrescor['feat nn_h1 recall0']=dfrescor['feat nn_h1 recall0'].apply(getcorinedesc,args=(dfcorcats,))
dfrescor['feat nn_h1 recall1']=dfrescor['feat nn_h1 recall1'].apply(getcorinedesc, args=(dfcorcats,))
dfrescor

,rank nn_h1 recall1,feat nn_h1 recall1,perm. imp. nn_h1 recall1,rank nn_h1 recall0,feat nn_h1 recall0,perm. imp. nn_h1 recall0
0,4,3.2.4 Transitional woodland-shrub,3.4%,4,2.4.2 Complex cultivation patterns,3.4%
1,5,3.2.3 Sclerophyllous vegetation,3.4%,5,2.2.3 Olive groves,3.2%
2,8,1.1.2 Discontinuous urban fabric,2.8%,6,3.3.4 Burnt areas,3.1%
3,10,3.1.2 Coniferous forest,2.3%,7,3.2.3 Sclerophyllous vegetation,3.0%
4,12,2.4.2 Complex cultivation patterns,1.7%,8,3.3.3 Sparsely vegetated areas,2.0%
5,13,2.4.3 Land principally occupied by agriculture...,1.7%,9,1.1.2 Discontinuous urban fabric,1.6%
6,14,1.1.1 Continuous urban fabric,1.6%,13,2.2.1 Vineyards,1.1%
7,16,3.3.3 Sparsely vegetated areas,1.1%,14,1.2.1 Industrial or commercial units,1.0%
8,17,1.2.1 Industrial or commercial units,1.0%,17,3.1.1 Broad-leaved forest,0.7%
9,18,2.1.1 Non-irrigated arable land,0.9%,18,2.3.1 Pastures,0.7%
